### 라이브러리 imports

In [ ]:
import pandas as pd
import numpy as np
import folium
import json
import requests

## 문제

대전의 인구수를 choropleth map을 이용하여 구별로 시각화 하고 결과를 화면캡쳐 후 제출하라.

1. 아래를 `geo_data`로 이용할 것

In [7]:
local_dict = json.loads(requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-municipalities-2018-geo.json').text)
daejeon_dict = local_dict.copy() 
daejeon_dict['features'] = [l for l in local_dict['features'] if l['properties']['code'][:2] == '25']  ## 앞자리가 25로 시작하는 게 대전인가봄.

2. 아래를 `data`로 이용할 것

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/2021-11-22-muni.csv')[67:72].reset_index(drop=True)
df

,행정구역(시군구)별,총인구수 (명)
0,동구,223384
1,중구,231852
2,서구,474046
3,유성구,350284
4,대덕구,174662


**hint**

* 대전의 (위도,경도)는 (36.34913,127.3849) 임.

**시각화 예시**

![](https://guebin.github.io/DV2023/posts/11wk-1_files/figure-html/2c649c68-a629-4431-bf76-ea1e17d2072a-1-cab33bcb-b895-4f5f-83f2-5414b2154ee7.png)

### 풀이

In [8]:
daejeon_dict['features'][0]['properties']

{'name': '동구', 'base_year': '2018', 'name_eng': 'Dong-gu', 'code': '25010'}

In [30]:
m = folium.Map(
    scrollWheelZoom = False,
    location = [36.34913,127.3849],
    zoom_start = 11
)

folium.Choropleth(
    geo_data = daejeon_dict,
    key_on = 'properties.name',
    data = df,
    columns = ['행정구역(시군구)별', '총인구수 (명)']
).add_to(m)

m

In [31]:
#!pip install pyppeteer
from folium import utilities
from pyppeteer import launch

# 만든 지도를 png 파일로 캡쳐해서 저장하는 함수
async def map_to_png(filename, m):
    html = m.get_root().render()
    browser = await launch(headless=True)

    page = await browser.newPage()
    with utilities.temp_html_filepath(html) as fname:
        await page.goto('file://{path}'.format(path=fname))

    img_data = await page.screenshot({'path' : 'C:/Users/hollyriver/Desktop/{}.png'.format(filename), 'fullPage': 'true', })
    await browser.close()

await map_to_png('지도', m)

**끝**